# In this section i will a lot of technics to create a recommandation system hybride 

In [9]:
import pymongo 
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import json
import pprint as p

In [10]:
books_brute = pd.read_csv('../data_set/BX-CSV-Dump/BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/home/ay/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshe

In [11]:
books_brute

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [12]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["SytemRecommandation"]
mycol = mydb["meta_book"]

In [13]:
books = []
for x in mycol.find({},{'_id':0, 'isbn':1, 'average_rating':1, 'book_id':1, 'description':1, 'num_pages':1, 'publisher':1, 'title':1, "publication_year":1, "format":1, "text_reviews_count":1, "ratings_count":1}):
    books.append(x)

In [14]:
books[0]

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'average_rating': '4.00',
 'description': '',
 'format': 'Paperback',
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_year': '1984',
 'book_id': '5333265',
 'ratings_count': '3',
 'title': 'W.C. Fields: A Life on Film'}

In [15]:
books_brute_ = pd.DataFrame(books, columns= ['isbn', 'text_reviews_count', 'average_rating', 'description', 'format', "publisher", 'num_pages', 'publication_year', 'book_id', 'ratings_count', 'title'])

In [16]:
books_brute_

,isbn,text_reviews_count,average_rating,description,format,publisher,num_pages,publication_year,book_id,ratings_count,title
0,0312853122,1,4.00,,Paperback,St. Martin's Press,256,1984,5333265,3,W.C. Fields: A Life on Film
1,0850308712,5,3.40,,,,,,287140,15,Runic Astrology: Starcraft and Timekeeping in ...
2,0743294297,3282,3.49,Addie Downs and Valerie Adler were eight when ...,Hardcover,Atria Books,368,2009,6066819,51184,Best Friends Forever
3,,7,4.03,Omnibus book club edition containing the Ladie...,Hardcover,"Nelson Doubleday, Inc.",600,1987,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."
4,0743509986,6,3.23,"Anita Diamant's international bestseller ""The ...",Audio CD,Simon & Schuster Audio,,2001,1333909,10,Good Harbor
...,...,...,...,...,...,...,...,...,...,...,...
664995,1847679277,113,3.84,The companion guide to the hysterical televisi...,Paperback,Canongate UK,225,2012,11165674,868,An Idiot Abroad: The Travel Diaries of Karl Pi...
664996,,6,4.60,This installment is the conclusion to part 1 o...,,,,,23643562,30,Tyce & Angelique: The Conclusion (The Tyce Ser...
664997,1501105035,84,4.03,New York Times bestselling author and award-wi...,Hardcover,Touchstone,192,2015,23492545,633,Move Your Bus: An Extraordinary New Approach t...
664998,0739033336,1,3.71,"With Over 40,000 words including slang and hip...",Paperback,Firebrand Music,183,2003,727224,7,"Hip-Hop Rhyming Dictionary: For Rappers, Djs a..."


In [17]:
# books_brute_.drop(columns=['publisher', 'publication_year', 'title'], inplace=True)
books_brute_.rename({'isbn':'ISBN'}, axis='columns', inplace=True)
books_brute_

,ISBN,text_reviews_count,average_rating,description,format,publisher,num_pages,publication_year,book_id,ratings_count,title
0,0312853122,1,4.00,,Paperback,St. Martin's Press,256,1984,5333265,3,W.C. Fields: A Life on Film
1,0850308712,5,3.40,,,,,,287140,15,Runic Astrology: Starcraft and Timekeeping in ...
2,0743294297,3282,3.49,Addie Downs and Valerie Adler were eight when ...,Hardcover,Atria Books,368,2009,6066819,51184,Best Friends Forever
3,,7,4.03,Omnibus book club edition containing the Ladie...,Hardcover,"Nelson Doubleday, Inc.",600,1987,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."
4,0743509986,6,3.23,"Anita Diamant's international bestseller ""The ...",Audio CD,Simon & Schuster Audio,,2001,1333909,10,Good Harbor
...,...,...,...,...,...,...,...,...,...,...,...
664995,1847679277,113,3.84,The companion guide to the hysterical televisi...,Paperback,Canongate UK,225,2012,11165674,868,An Idiot Abroad: The Travel Diaries of Karl Pi...
664996,,6,4.60,This installment is the conclusion to part 1 o...,,,,,23643562,30,Tyce & Angelique: The Conclusion (The Tyce Ser...
664997,1501105035,84,4.03,New York Times bestselling author and award-wi...,Hardcover,Touchstone,192,2015,23492545,633,Move Your Bus: An Extraordinary New Approach t...
664998,0739033336,1,3.71,"With Over 40,000 words including slang and hip...",Paperback,Firebrand Music,183,2003,727224,7,"Hip-Hop Rhyming Dictionary: For Rappers, Djs a..."


In [18]:
books_brute.drop(columns=['Image-URL-S', 'Image-URL-M'], inplace=True)

In [19]:
books_brute

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...


In [20]:
books_brute.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-L            3
dtype: int64

In [21]:
df_merge = pd.merge(books_brute, books_brute_, on='ISBN')

In [22]:
df_merge

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,text_reviews_count,average_rating,description,format,publisher,num_pages,publication_year,book_id,ratings_count,title
0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,8,4.18,,,,,,1357556,34,The Mummies of Ürümchi
1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,Berkley Books,395,,406281,1692,What If?: The World's Foremost Historians Imag...
2,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,2,3.68,"For this rousing, revisionist history, the for...",Hardcover,Random House,317,1996,817876,58,Under the Black Flag: The Romance and the Real...
3,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,113,4.18,The Onionhas quickly become the world's most p...,Paperback,Three Rivers Press,164,1999,579696,11158,Our Dumb Century: The Onion Presents 100 Years...
4,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,http://images.amazon.com/images/P/1879384493.0...,2,3.28,If I'd Known Then What I Know Now is an inspir...,Paperback,Cypress House,136,2003,302816,18,If I'd Known Then What I Know Now: Why Not Lea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31651,067975475X,Skinned Alive: Stories,Edmund White,1996,Vintage Books USA,http://images.amazon.com/images/P/067975475X.0...,15,3.71,The eight stories in this erotic and heartbrea...,Paperback,Vintage,288,1996,109723,197,Skinned Alive
31652,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,1998,Addison-Wesley Professional,http://images.amazon.com/images/P/020130998X.0...,4,3.87,"James Rumbaugh, Ivar Jacobson, and Grady Booch...",Hardcover,Addison-Wesley Professional,576,1998,1920174,39,The Unified Modeling Language Reference Manual
31653,0684860112,Driving to Detroit: Memoirs of a Fast Woman,Lesley Hazleton,1999,Simon &amp; Schuster (Trade Division),http://images.amazon.com/images/P/0684860112.0...,3,4.00,,,,,,1058862,26,Driving To Detroit: Memoirs Of A Fast Woman
31654,3442150663,Der Mossad.,Victor Ostrovsky,2000,Goldmann,http://images.amazon.com/images/P/3442150663.0...,2,3.89,The #1 NY Times bestseller the Israeli foreign...,,,,,333662,26,Der Mossad


In [23]:
df_merge.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-L            0
text_reviews_count     0
average_rating         0
description            0
format                 0
publisher              0
num_pages              0
publication_year       0
book_id                0
ratings_count          0
title                  0
dtype: int64

In [24]:
mycol_ = mydb["genres_book"]

In [25]:
genres_books = []
for x in mycol_.find({},{'_id':0}):
    genres_books.append(x)

In [26]:
df_genres_books = pd.DataFrame(genres_books, columns=['book_id', 'genres'])

In [27]:
df_final = pd.merge(df_merge, df_genres_books, on='book_id')
df_final

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,text_reviews_count,average_rating,description,format,publisher,num_pages,publication_year,book_id,ratings_count,title,genres
0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,8,4.18,,,,,,1357556,34,The Mummies of Ürümchi,"{'history, historical fiction, biography': 65,..."
1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,Berkley Books,395,,406281,1692,What If?: The World's Foremost Historians Imag...,"{'non-fiction': 116, 'history, historical fict..."
2,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,2,3.68,"For this rousing, revisionist history, the for...",Hardcover,Random House,317,1996,817876,58,Under the Black Flag: The Romance and the Real...,"{'history, historical fiction, biography': 779..."
3,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,113,4.18,The Onionhas quickly become the world's most p...,Paperback,Three Rivers Press,164,1999,579696,11158,Our Dumb Century: The Onion Presents 100 Years...,"{'fiction': 58, 'history, historical fiction, ..."
4,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,http://images.amazon.com/images/P/1879384493.0...,2,3.28,If I'd Known Then What I Know Now is an inspir...,Paperback,Cypress House,136,2003,302816,18,If I'd Known Then What I Know Now: Why Not Lea...,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31651,067975475X,Skinned Alive: Stories,Edmund White,1996,Vintage Books USA,http://images.amazon.com/images/P/067975475X.0...,15,3.71,The eight stories in this erotic and heartbrea...,Paperback,Vintage,288,1996,109723,197,Skinned Alive,"{'fiction': 30, 'romance': 2, 'history, histor..."
31652,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,1998,Addison-Wesley Professional,http://images.amazon.com/images/P/020130998X.0...,4,3.87,"James Rumbaugh, Ivar Jacobson, and Grady Booch...",Hardcover,Addison-Wesley Professional,576,1998,1920174,39,The Unified Modeling Language Reference Manual,{}
31653,0684860112,Driving to Detroit: Memoirs of a Fast Woman,Lesley Hazleton,1999,Simon &amp; Schuster (Trade Division),http://images.amazon.com/images/P/0684860112.0...,3,4.00,,,,,,1058862,26,Driving To Detroit: Memoirs Of A Fast Woman,{'non-fiction': 2}
31654,3442150663,Der Mossad.,Victor Ostrovsky,2000,Goldmann,http://images.amazon.com/images/P/3442150663.0...,2,3.89,The #1 NY Times bestseller the Israeli foreign...,,,,,333662,26,Der Mossad,"{'non-fiction': 51, 'history, historical ficti..."


In [32]:
# df_final['genres'] = df_final['genres'].apply(lambda x: " ".join(set(list(x))))
df_final['genre'] = df_final['genres'].astype('str').apply(lambda x: x.split(',')[0])

In [33]:
df_final

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,text_reviews_count,average_rating,description,format,publisher,num_pages,publication_year,book_id,ratings_count,title,genres,genre
0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,8,4.18,,,,,,1357556,34,The Mummies of Ürümchi,"non-fiction history, historical fiction, biogr...",non-fiction history
1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,Berkley Books,395,,406281,1692,What If?: The World's Foremost Historians Imag...,"fiction fantasy, paranormal non-fiction histor...",fiction fantasy
2,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,2,3.68,"For this rousing, revisionist history, the for...",Hardcover,Random House,317,1996,817876,58,Under the Black Flag: The Romance and the Real...,"fiction mystery, thriller, crime non-fiction h...",fiction mystery
3,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,113,4.18,The Onionhas quickly become the world's most p...,Paperback,Three Rivers Press,164,1999,579696,11158,Our Dumb Century: The Onion Presents 100 Years...,"fiction comics, graphic non-fiction history, h...",fiction comics
4,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,http://images.amazon.com/images/P/1879384493.0...,2,3.28,If I'd Known Then What I Know Now is an inspir...,Paperback,Cypress House,136,2003,302816,18,If I'd Known Then What I Know Now: Why Not Lea...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31651,067975475X,Skinned Alive: Stories,Edmund White,1996,Vintage Books USA,http://images.amazon.com/images/P/067975475X.0...,15,3.71,The eight stories in this erotic and heartbrea...,Paperback,Vintage,288,1996,109723,197,Skinned Alive,"fiction non-fiction poetry romance history, hi...",fiction non-fiction poetry romance history
31652,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,1998,Addison-Wesley Professional,http://images.amazon.com/images/P/020130998X.0...,4,3.87,"James Rumbaugh, Ivar Jacobson, and Grady Booch...",Hardcover,Addison-Wesley Professional,576,1998,1920174,39,The Unified Modeling Language Reference Manual,,
31653,0684860112,Driving to Detroit: Memoirs of a Fast Woman,Lesley Hazleton,1999,Simon &amp; Schuster (Trade Division),http://images.amazon.com/images/P/0684860112.0...,3,4.00,,,,,,1058862,26,Driving To Detroit: Memoirs Of A Fast Woman,non-fiction,non-fiction
31654,3442150663,Der Mossad.,Victor Ostrovsky,2000,Goldmann,http://images.amazon.com/images/P/3442150663.0...,2,3.89,The #1 NY Times bestseller the Israeli foreign...,,,,,333662,26,Der Mossad,"fiction mystery, thriller, crime non-fiction h...",fiction mystery


In [36]:
# for i in range(len(df_final['genres'].values)):
#     df_final.loc[i, 'genres'] = " ".join(set(list(df_final['genres'][i])))
df_final

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,text_reviews_count,average_rating,description,format,num_pages,book_id,ratings_count,genres
0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,4.18,,,,1357556,34,"non-fiction history, historical fiction, biogr..."
1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,395,406281,1692,"fiction fantasy, paranormal non-fiction histor..."
2,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,2,3.68,"For this rousing, revisionist history, the for...",Hardcover,317,817876,58,"mystery, thriller, crime fiction non-fiction h..."
3,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,113,4.18,The Onionhas quickly become the world's most p...,Paperback,164,579696,11158,"fiction non-fiction comics, graphic history, h..."
4,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,2,3.28,If I'd Known Then What I Know Now is an inspir...,Paperback,136,302816,18,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
31651,067975475X,Skinned Alive: Stories,Edmund White,1996,Vintage Books USA,15,3.71,The eight stories in this erotic and heartbrea...,Paperback,288,109723,197,"fiction history, historical fiction, biography..."
31652,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,1998,Addison-Wesley Professional,4,3.87,"James Rumbaugh, Ivar Jacobson, and Grady Booch...",Hardcover,576,1920174,39,
31653,0684860112,Driving to Detroit: Memoirs of a Fast Woman,Lesley Hazleton,1999,Simon &amp; Schuster (Trade Division),3,4.00,,,,1058862,26,non-fiction
31654,3442150663,Der Mossad.,Victor Ostrovsky,2000,Goldmann,2,3.89,The #1 NY Times bestseller the Israeli foreign...,,,333662,26,"fiction non-fiction mystery, thriller, crime h..."


In [65]:
# df_final['ratings_count'].hist();

In [38]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31656 entries, 0 to 31655
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ISBN                 31656 non-null  object
 1   Book-Title           31656 non-null  object
 2   Book-Author          31656 non-null  object
 3   Year-Of-Publication  31656 non-null  object
 4   Publisher            31656 non-null  object
 5   text_reviews_count   31656 non-null  object
 6   average_rating       31656 non-null  object
 7   description          31656 non-null  object
 8   format               31656 non-null  object
 9   num_pages            31656 non-null  object
 10  book_id              31656 non-null  object
 11  ratings_count        31656 non-null  object
 12  genres               31656 non-null  object
dtypes: object(13)
memory usage: 4.4+ MB


In [40]:
df_final['genres'].nunique()

625

In [42]:
df_final['format'].nunique()

48

In [43]:
df_final['format'].unique()

array(['', 'Hardcover', 'Paperback', 'Mass Market Paperback',
       'Unknown Binding', 'Audiobook', 'Trade Paperback', 'capa mole',
       'Audio CD', 'Taschenbuch', 'paper', 'Library Binding', 'paperback',
       'Leather Bound', 'Audio', 'Kindle Edition', 'ebook',
       'Spiral-bound', 'Audio Cassette', 'Board Book', 'Novelty Book',
       'Board book', 'Ring-bound', 'Mass paperback', 'Casebound',
       'Tradepaperback', 'Poche', 'Broche', 'Mass Market', 'Capa Mole',
       'Pocket Paperback', 'Bantam New Fiction', 'Broschiert', 'cloth',
       'Gebundene Ausgabe', 'Perfect Paperback', 'Board',
       'Audio Cassette - Abridged', 'Plastic Comb', 'hardback',
       'Hardcover, Large Print', 'audio cd', 'Hardcover 1st edition',
       'Graphic Novel', 'audio cassette', 'School &amp; Library Binding',
       'hardcover', 'softcovers'], dtype=object)

## la recommandation simple 

# section 1

In [38]:
# df_final.to_csv('data_set_book_merge.csv')
import pandas as pd
# df_final = pd.read_csv('data_set_book_merge.csv')
df_final = df_final.drop(columns=['publisher'])
df_final.to_csv('data_set_book_merge_.csv')

In [39]:
df_final.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,text_reviews_count,average_rating,description,format,num_pages,publication_year,book_id,ratings_count,title,genre
0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,8,4.18,,,,,1357556,34,The Mummies of Ürümchi,non-fiction history
1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,395,,406281,1692,What If?: The World's Foremost Historians Imag...,fiction fantasy


In [ ]:
set(df_final['ISBN'])

### A/ Utilisation de la formule des notes pondérées (Weighted ratings)

Weighted Rating (WR) =  ((v/(v+m)).R)+((m/(v+m)).C)

v: est le nombre de votes 

m: est le nombre de votes minimum requis pour figurer dans le tableau

R: est la note moyenne 

C: est le vote moyen sur l'ensemble du rapport

In [6]:
# Nombre de votes pour livre
v = df_final[df_final['ratings_count'].notnull()]['ratings_count'].astype('int')
v

0           34
1         1692
2           58
3        11158
4           18
         ...  
31651      197
31652       39
31653       26
31654       26
31655       13
Name: ratings_count, Length: 31656, dtype: int64

In [7]:
# Note Moyenne livre
r = df_final[df_final['average_rating'].notnull()]['average_rating'].astype('float')
r

0        4.18
1        3.68
2        3.68
3        4.18
4        3.28
         ... 
31651    3.71
31652    3.87
31653    4.00
31654    3.89
31655    3.23
Name: average_rating, Length: 31656, dtype: float64

In [8]:
 # Note Moyenne sur tout l'ensemble
C = r.mean()
C

3.8096957922668837

In [9]:
# le nombre de votes minimum pour etre dans la liste finale 
m = v.quantile(.95)
m

4412.25

In [10]:
# Creation de la liste finale des films
finalList = df_final[(df_final['ratings_count'].astype('float') >= m) & (df_final['ratings_count'].notnull()) & (df_final['average_rating'].notnull())][['ISBN', 'Book-Title', "ratings_count", "average_rating", 'Book-Author', 'Year-Of-Publication', 'Publisher', 'genres']]
finalList['ratings_count'] = finalList['ratings_count'].astype('int')
finalList['average_rating'] = finalList['average_rating'].astype('float')
finalList

,ISBN,Book-Title,ratings_count,average_rating,Book-Author,Year-Of-Publication,Publisher,genres
3,0609804618,Our Dumb Century: The Onion Presents 100 Years...,11158,4.18,The Onion,1999,Three Rivers Press,"fiction non-fiction comics, graphic history, h..."
10,0553582909,Icebound,15550,3.73,Dean R. Koontz,2000,Bantam Books,"mystery, thriller, crime fiction fantasy, para..."
18,0345465083,Seabiscuit,113228,4.20,LAURA HILLENBRAND,2003,Ballantine Books,"fiction non-fiction history, historical fictio..."
26,0446612545,The Beach House,69256,3.82,James Patterson,2003,Warner Books,"mystery, thriller, crime fiction romance"
27,0671027360,Angels &amp; Demons,9136,3.86,Dan Brown,2001,Pocket Star,"fiction fantasy, paranormal history, historica..."
...,...,...,...,...,...,...,...,...
31471,0142300578,"Changeling (Sweep, 8)",7635,4.18,Cate Tiernan,2001,Puffin Books,"fiction fantasy, paranormal mystery, thriller,..."
31472,0142300861,"Reckoning (Sweep, 13)",5737,4.13,Cate Tiernan,2002,Puffin Books,"fiction fantasy, paranormal mystery, thriller,..."
31494,0896086283,Feminism Is for Everybody: Passionate Politics,7740,4.18,bell hooks,2000,South End Press,"non-fiction history, historical fiction, biogr..."
31608,0140296395,Atop an Underwood,4919,3.52,Jack Kerouac,2000,Penguin Books,"fiction history, historical fiction, biography..."


In [11]:
max(finalList['average_rating'].values)*2 , min(finalList['average_rating'].values)*2

(9.42, 5.6)

In [12]:
#Top livres selon leurs note ponderée.

In [13]:
def weighted_rating(x):
    v = x['ratings_count']
    R = x['average_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)
finalList['weighted ratings'] = finalList.apply(weighted_rating, axis=1)
finalList = finalList.sort_values('weighted ratings', ascending=False)

In [14]:
# Top livres selon score
finalList

,ISBN,Book-Title,ratings_count,average_rating,Book-Author,Year-Of-Publication,Publisher,genres,weighted ratings
2782,0836218051,The Essential Calvin and Hobbes,95116,4.64,Bill Watterson,1988,Andrews McMeel Publishing,"fiction fantasy, paranormal children non-ficti...",4.603191
2781,0836217691,Homicidal Psycho Jungle Cat: A Calvin and Hobb...,14276,4.71,Bill Watterson,1994,Andrews McMeel Publishing,"fiction fantasy, paranormal children comics, g...",4.497440
21062,0553328255,The Complete Sherlock Holmes: All 4 Novels and...,111473,4.50,Sir Arthur Conan Doyle,1986,Bantam Classics,"fiction fantasy, paranormal mystery, thriller,...",4.473717
6087,0525467262,The Complete Tales &amp; Poems of Winnie-The-Pooh,37743,4.49,A. A. Milne,2001,Dutton Books,"fiction fantasy, paranormal poetry children yo...",4.418795
19179,0618346260,"The Two Towers (The Lord of the Rings, Part 2)",490005,4.42,J. R. R. Tolkien,2003,Houghton Mifflin Company,"fiction fantasy, paranormal young-adult",4.414554
...,...,...,...,...,...,...,...,...,...
5444,0192834401,The Castle of Otranto: A Gothic Story (Oxford ...,13331,3.18,Horace Walpole,1998,Oxford University Press,"fiction fantasy, paranormal history, historica...",3.336588
1250,0452281423,The Claiming of Sleeping Beauty (Sleeping Beauty),38843,3.28,A.N. Roquelaure,1999,Plume Books,"fiction history, historical fiction, biography...",3.334032
2259,0743431014,Songs of the Humpback Whale : A Novel in Five ...,25003,3.19,Jodi Picoult,2001,Washington Square Press,fiction young-adult,3.282954
1872,0060988649,Lost: A Novel,12607,2.81,Gregory Maguire,2002,Regan Books,"fiction fantasy, paranormal history, historica...",3.069172


In [31]:
data_genres = df_final.copy()
data_genres['genre'] = df_final['genres'].astype('str').apply(lambda x: x.split(',')[0])
data_genres

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,text_reviews_count,average_rating,description,format,publisher,num_pages,publication_year,book_id,ratings_count,title,genres,genre
0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,8,4.18,,,,,,1357556,34,The Mummies of Ürümchi,"non-fiction history, historical fiction, biogr...",non-fiction history
1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,http://images.amazon.com/images/P/0425176428.0...,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,Berkley Books,395,,406281,1692,What If?: The World's Foremost Historians Imag...,"fiction fantasy, paranormal non-fiction histor...",fiction fantasy
2,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,http://images.amazon.com/images/P/0679425608.0...,2,3.68,"For this rousing, revisionist history, the for...",Hardcover,Random House,317,1996,817876,58,Under the Black Flag: The Romance and the Real...,"fiction mystery, thriller, crime non-fiction h...",fiction mystery
3,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,http://images.amazon.com/images/P/0609804618.0...,113,4.18,The Onionhas quickly become the world's most p...,Paperback,Three Rivers Press,164,1999,579696,11158,Our Dumb Century: The Onion Presents 100 Years...,"fiction comics, graphic non-fiction history, h...",fiction comics
4,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,http://images.amazon.com/images/P/1879384493.0...,2,3.28,If I'd Known Then What I Know Now is an inspir...,Paperback,Cypress House,136,2003,302816,18,If I'd Known Then What I Know Now: Why Not Lea...,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31651,067975475X,Skinned Alive: Stories,Edmund White,1996,Vintage Books USA,http://images.amazon.com/images/P/067975475X.0...,15,3.71,The eight stories in this erotic and heartbrea...,Paperback,Vintage,288,1996,109723,197,Skinned Alive,"fiction non-fiction poetry romance history, hi...",fiction non-fiction poetry romance history
31652,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,1998,Addison-Wesley Professional,http://images.amazon.com/images/P/020130998X.0...,4,3.87,"James Rumbaugh, Ivar Jacobson, and Grady Booch...",Hardcover,Addison-Wesley Professional,576,1998,1920174,39,The Unified Modeling Language Reference Manual,,
31653,0684860112,Driving to Detroit: Memoirs of a Fast Woman,Lesley Hazleton,1999,Simon &amp; Schuster (Trade Division),http://images.amazon.com/images/P/0684860112.0...,3,4.00,,,,,,1058862,26,Driving To Detroit: Memoirs Of A Fast Woman,non-fiction,non-fiction
31654,3442150663,Der Mossad.,Victor Ostrovsky,2000,Goldmann,http://images.amazon.com/images/P/3442150663.0...,2,3.89,The #1 NY Times bestseller the Israeli foreign...,,,,,333662,26,Der Mossad,"fiction mystery, thriller, crime non-fiction h...",fiction mystery


### C/ Top book selon le genre

In [15]:
data_genres = df_final.copy()
for i in range(len(df_final['genres'].values)):
    data_genres.loc[i, 'genre'] = df_final['genres'].astype('str')[i].split(',')[0]
data_genres

,Unnamed: 0,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,text_reviews_count,average_rating,description,format,num_pages,book_id,ratings_count,genres,genre
0,0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,4.18,NaN,NaN,NaN,1357556,34,"non-fiction history, historical fiction, biogr...",non-fiction history
1,1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,395.0,406281,1692,"fiction fantasy, paranormal non-fiction histor...",fiction fantasy
2,2,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,2,3.68,"For this rousing, revisionist history, the for...",Hardcover,317.0,817876,58,"mystery, thriller, crime fiction non-fiction h...",mystery
3,3,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999,Three Rivers Press,113,4.18,The Onionhas quickly become the world's most p...,Paperback,164.0,579696,11158,"fiction non-fiction comics, graphic history, h...",fiction non-fiction comics
4,4,1879384493,If I'd Known Then What I Know Now: Why Not Lea...,J. R. Parrish,2003,Cypress House,2,3.28,If I'd Known Then What I Know Now is an inspir...,Paperback,136.0,302816,18,NaN,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31651,31651,067975475X,Skinned Alive: Stories,Edmund White,1996,Vintage Books USA,15,3.71,The eight stories in this erotic and heartbrea...,Paperback,288.0,109723,197,"fiction history, historical fiction, biography...",fiction history
31652,31652,020130998X,The Unified Modeling Language Reference Manual...,James Rumbaugh,1998,Addison-Wesley Professional,4,3.87,"James Rumbaugh, Ivar Jacobson, and Grady Booch...",Hardcover,576.0,1920174,39,NaN,nan
31653,31653,0684860112,Driving to Detroit: Memoirs of a Fast Woman,Lesley Hazleton,1999,Simon &amp; Schuster (Trade Division),3,4.00,NaN,NaN,NaN,1058862,26,non-fiction,non-fiction
31654,31654,3442150663,Der Mossad.,Victor Ostrovsky,2000,Goldmann,2,3.89,The #1 NY Times bestseller the Israeli foreign...,NaN,NaN,333662,26,"fiction non-fiction mystery, thriller, crime h...",fiction non-fiction mystery


In [16]:
data_genres.isnull().sum()

Unnamed: 0                0
ISBN                      0
Book-Title                0
Book-Author               0
Year-Of-Publication       0
Publisher                 0
text_reviews_count        0
average_rating            0
description            4407
format                 5532
num_pages              6485
book_id                   0
ratings_count             0
genres                  781
genre                     0
dtype: int64

In [17]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31656 entries, 0 to 31655
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           31656 non-null  int64  
 1   ISBN                 31656 non-null  object 
 2   Book-Title           31656 non-null  object 
 3   Book-Author          31656 non-null  object 
 4   Year-Of-Publication  31656 non-null  int64  
 5   Publisher            31656 non-null  object 
 6   text_reviews_count   31656 non-null  int64  
 7   average_rating       31656 non-null  float64
 8   description          27249 non-null  object 
 9   format               26124 non-null  object 
 10  num_pages            25171 non-null  float64
 11  book_id              31656 non-null  int64  
 12  ratings_count        31656 non-null  int64  
 13  genres               30875 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 3.4+ MB


In [18]:
# data_genres.drop(columns=['genres'], inplace=True)

In [19]:
data_genres.head(3)

,Unnamed: 0,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,text_reviews_count,average_rating,description,format,num_pages,book_id,ratings_count,genres,genre
0,0,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,8,4.18,NaN,NaN,NaN,1357556,34,"non-fiction history, historical fiction, biogr...",non-fiction history
1,1,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,113,3.68,Historians and inquisitive laymen alike love t...,Hardcover,395.0,406281,1692,"fiction fantasy, paranormal non-fiction histor...",fiction fantasy
2,2,0679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,2,3.68,"For this rousing, revisionist history, the for...",Hardcover,317.0,817876,58,"mystery, thriller, crime fiction non-fiction h...",mystery


In [20]:
# je vais creer une fonction qui retoure les Tops liver selon le genre
def Top_by_genre(genre, percentile=0.85):
    df_local = data_genres[data_genres['genre'] == genre]
    vote_counts = df_local[df_local['ratings_count'].notnull()]['ratings_count'].astype('int')
    vote_averages = df_local[df_local['average_rating'].notnull()]['average_rating'].astype('float')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    finalList = df_local[(df_local['ratings_count'].astype('int') >= m) & (df_local['ratings_count'].notnull()) & (df_local['average_rating'].notnull())][['ISBN', 'Book-Title', "ratings_count", "average_rating", 'Book-Author', 'Year-Of-Publication', 'Publisher', 'genre']]
    finalList['ratings_count'] = finalList['ratings_count'].astype('int')
    finalList['average_rating'] = finalList['average_rating'].astype('float')
    
    finalList['weighted ratings'] = finalList.apply(lambda x: (x['ratings_count']/(x['ratings_count']+m) * x['average_rating']) + (m/(m+x['ratings_count']) * C), axis=1)
    finalList = finalList.sort_values('weighted ratings', ascending=False)
    return finalList
    

In [21]:
# le nombre de votes minimum pour etre pris en consideration dans la liste retourné par Top_by_genre()
m = v.quantile(.85)
m

941.0

In [22]:
Top_by_genre('mystery').head(10)

,ISBN,Book-Title,ratings_count,average_rating,Book-Author,Year-Of-Publication,Publisher,genre,weighted ratings
17858,0451459873,Blood Rites (The Dresden Files Series Book 6),81169,4.33,Jim Butcher,2004,Roc,mystery,4.324281
3441,0767900316,The Corner : A Year in the Life of an Inner-Ci...,3886,4.41,DAVID SIMON,1998,Broadway,mystery,4.296522
1010,0451458923,"Summer Knight (The Dresden Files, Book 4)",92791,4.30,Jim Butcher,2002,Roc,mystery,4.295259
1166,0425178579,Betrayal in Death,19569,4.31,Nora Roberts,2001,Berkley Publishing Group,mystery,4.287837
318,0446611239,The Cabinet of Curiosities,29529,4.26,Douglas Preston,2003,Warner Books,mystery,4.246486
18632,0688172172,Phantoms in the Brain : Probing the Mysteries ...,12634,4.25,V. S. Ramachandran,1999,Perennial,mystery,4.220159
1423,0553212427,Sherlock Holmes: The Complete Novels and Stori...,1346,4.50,Arthur Conan Doyle,1986,Bantam,mystery,4.220000
8133,0345451902,The Last Detective (Elvis Cole Novels (Paperba...,12335,4.24,ROBERT CRAIS,2004,Ballantine Books,mystery,4.210117
4793,0394757688,The Long Goodbye (Vintage Crime/Black Lizard),22913,4.22,Raymond Chandler,1992,Vintage Books USA,mystery,4.204132
2664,0440226457,Time to Hunt: A Novel,7381,4.25,Stephen Hunter,1999,Delta Trade Paperbacks,mystery,4.201086


In [23]:
Top_by_genre('fiction fantasy').head(10)

,ISBN,Book-Title,ratings_count,average_rating,Book-Author,Year-Of-Publication,Publisher,genre,weighted ratings
2782,0836218051,The Essential Calvin and Hobbes,95116,4.64,Bill Watterson,1988,Andrews McMeel Publishing,fiction fantasy,4.625932
2781,0836217691,Homicidal Psycho Jungle Cat: A Calvin and Hobb...,14276,4.71,Bill Watterson,1994,Andrews McMeel Publishing,fiction fantasy,4.617304
21062,0553328255,The Complete Sherlock Holmes: All 4 Novels and...,111473,4.50,Sir Arthur Conan Doyle,1986,Bantam Classics,fiction fantasy,4.490085
6087,0525467262,The Complete Tales &amp; Poems of Winnie-The-Pooh,37743,4.49,A. A. Milne,2001,Dutton Books,fiction fantasy,4.461993
7070,1570820872,Disney's the Lion King (Disney Classic Series),19856,4.50,Don Ferguson,1994,Random House Childrens Pub,fiction fantasy,4.447974
22807,0517219026,"Stephen King: Three Complete Novels: Carrie, '...",11256,4.52,Stephen King,2002,Wings,fiction fantasy,4.430834
2819,1563892251,"The Doll's House (Sandman, Book 2)",55387,4.44,Neil Gaiman,1999,DC Comics,fiction fantasy,4.422148
19179,0618346260,"The Two Towers (The Lord of the Rings, Part 2)",490005,4.42,J. R. R. Tolkien,2003,Houghton Mifflin Company,fiction fantasy,4.417997
13745,0451933028,The Green Mile: The Complete Six Part Novel,177376,4.42,Stephen King,1996,Signet Book,fiction fantasy,4.414500
11930,0811200124,Labyrinths: Selected Stories and Other Writing...,20553,4.46,Jorge Luis Borges,1964,New Directions Publishing Corporation,fiction fantasy,4.412682


### recommandation base sur la description d'un livre

In [24]:
df_final['description'].fillna('', inplace=True)

In [25]:
# from spacy_langdetect import LanguageDetector
# import spacy

In [26]:
# !pip install spacy_langdetect

In [27]:
# !pip install spacy

In [28]:
# nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [29]:
# nlp = spacy.load('en_core_web_sm')

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df_final['description'])
tfidf_matrix.shape

(31656, 1401742)

### Similitude des cosinus
la Similarité Cosinus est pour calculer une quantité numérique qui indique la similarité entre deux livres. Mathématiquement, elle est définie comme suit :
cosinus(x,y)=x.y⊺||x|||.||y|| 

In [31]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(31656, 31656)

In [32]:
cosine_sim[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
df_final = df_final.reset_index()
titles = df_final['Book-Title']
indices = pd.Series(df_final.index, index=df_final['Book-Title'])

In [34]:
indices

Book-Title
The Mummies of Urumchi                                                                                                            0
What If?: The World's Foremost Military Historians Imagine What Might Have Been                                                   1
Under the Black Flag: The Romance and the Reality of Life Among the Pirates                                                       2
Our Dumb Century: The Onion Presents 100 Years of Headlines from America's Finest News Source                                     3
If I'd Known Then What I Know Now: Why Not Learn from the Mistakes of Others? : You Can't Afford to Make Them All Yourself        4
                                                                                                                              ...  
Skinned Alive: Stories                                                                                                        31651
The Unified Modeling Language Reference Manual (Addison-Wesley Ob

In [35]:
titles[:16]

0                                The Mummies of Urumchi
1     What If?: The World's Foremost Military Histor...
2     Under the Black Flag: The Romance and the Real...
3     Our Dumb Century: The Onion Presents 100 Years...
4     If I'd Known Then What I Know Now: Why Not Lea...
5                                           Wild Animus
6                                              Airframe
7                              OUT OF THE SILENT PLANET
8                                     Sturmzeit. Roman.
9                                        Pigs in Heaven
10                                             Icebound
11                                      Purity in Death
12                                              Proxies
13                                    The Street Lawyer
14                           A Soldier of the Great War
15    The Dragons of Eden: Speculations on the Evolu...
Name: Book-Title, dtype: object

In [36]:
# une fonction qui renvoie les 15 livres les plus similaires sur la base du score de similarité cosinusoïdale.
def livre_recommande_selon(title):
    index = indices[title]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:16]
    livre_indices = [i[0] for i in sim_scores]
    print(livre_indices)
    return titles.iloc[livre_indices]

In [37]:
livre_recommande_selon('Purity in Death')

[1629, 4512, 13075, 3488, 14052, 29403, 3154, 20609, 19003, 8126, 17387, 22274, 8258, 21577, 23107]


1629     Imitation in Death (Eve Dallas Mysteries (Pape...
4512                                    The Fallen Sparrow
13075                                     Divided in Death
3488     Cat on a Hyacinth Hunt : A Midnight Louie Myst...
14052          Cat in a Kiwi Con: A Midnight Louie Mystery
29403                  Pussyfoot: A Midnight Louis Mystery
3154                                 Dare Truth or Promise
20609                                 What's Under My Bed?
19003                                         ZEN COMPUTER
8126               Net Bandits (Internet Detectives, No 1)
17387                                The Lady or the Tiger
22274                                Aberystwyth Mon Amour
8258     Ceremony in Death (Eve Dallas Mysteries (Paper...
21577    Mozart's Brain and the Fighter Pilot: Unleashi...
23107    The Second Brain : The Scientific Basis of Gut...
Name: Book-Title, dtype: object

### Ajout de Popularity et Ratings 


In [38]:
def recommendationsAmeliorees(title):
    index = indices[title]
    sim_scores = list(enumerate(cosine_sim[index]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    livre_indices = [i[0] for i in sim_scores]
    
    livres = df_final.iloc[livre_indices][['ISBN', 'Book-Title', "ratings_count", "average_rating", 'Book-Author', 'Year-Of-Publication', 'Publisher']]
    vote_counts = livres[livres['ratings_count'].notnull()]['ratings_count'].astype('int')
    vote_averages = livres[livres['average_rating'].notnull()]['average_rating'].astype('float')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    
    finalList = livres[(livres['ratings_count'].astype('int') >= m) & (livres['ratings_count'].notnull()) & (livres['average_rating'].notnull())]
    finalList['ratings_count'] = finalList['ratings_count'].astype('int')
    finalList['average_rating'] = finalList['average_rating'].astype('float')
    finalList['weighted ratings'] = finalList.apply(weighted_rating, axis=1)
    finalList = finalList.sort_values('weighted ratings', ascending=False).head(10)
    
    return finalList

In [39]:
recommendationsAmeliorees('Purity in Death')

<ipython-input-38-ae2953c935e4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalList['ratings_count'] = finalList['ratings_count'].astype('int')
<ipython-input-38-ae2953c935e4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalList['average_rating'] = finalList['average_rating'].astype('float')
<ipython-input-38-ae2953c935e4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,ISBN,Book-Title,ratings_count,average_rating,Book-Author,Year-Of-Publication,Publisher,weighted ratings
1166,0425178579,Betrayal in Death,19569,4.31,Nora Roberts,2001,Berkley Publishing Group,4.287046
1629,0425191583,Imitation in Death (Eve Dallas Mysteries (Pape...,851,4.34,J. D. Robb,2004,Berkley Publishing Group,4.061531
8258,0425157628,Ceremony in Death (Eve Dallas Mysteries (Paper...,1108,4.26,J. D. Robb,1997,Berkley Publishing Group,4.053199
3962,0375701079,"A User's Guide to the Brain: Perception, Atten...",1641,4.00,John J. Ratey,2002,Vintage Books USA,3.930644
17387,0874067987,The Lady or the Tiger,4118,3.94,Frank Stockton,1995,Worthington Publishing Company,3.915763
29403,0312852185,Pussyfoot: A Midnight Louis Mystery,633,3.99,Carole Nelson Douglas,1993,St Martins Pr,3.882207
3488,0812561864,Cat on a Hyacinth Hunt : A Midnight Louie Myst...,371,3.88,Carole Nelson Douglas,1999,Forge Books,3.829576
3154,0395971179,Dare Truth or Promise,4025,3.81,Paula Boock,1999,Houghton Mifflin Company,3.809942
13593,055356370X,The Hacker Crackdown: Law and Disorder on the ...,2399,3.80,Bruce Sterling,1993,Spectra Books,3.802732
21577,0609810057,Mozart's Brain and the Fighter Pilot: Unleashi...,446,3.59,"Richard M., Md. Restak",2002,Three Rivers Press (CA),3.739051


In [40]:
recommendationsAmeliorees('OUT OF THE SILENT PLANET')

<ipython-input-38-ae2953c935e4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalList['ratings_count'] = finalList['ratings_count'].astype('int')
<ipython-input-38-ae2953c935e4>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalList['average_rating'] = finalList['average_rating'].astype('float')
<ipython-input-38-ae2953c935e4>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,ISBN,Book-Title,ratings_count,average_rating,Book-Author,Year-Of-Publication,Publisher,weighted ratings
13218,068803036X,The Cider House Rules,712,4.10,John Irving,1985,Harpercollins Childrens Books,3.934739
4546,0553572393,Green Mars,25023,3.92,Kim Stanley Robinson,1995,Spectra Books,3.916002
5333,0553573357,Blue Mars,19911,3.92,Kim Stanley Robinson,1997,Spectra Books,3.915022
4160,0743234901,Out of the Silent Planet (Space Trilogy (Paper...,2845,3.92,C.S. Lewis,2003,Scribner,3.892584
14843,0553196480,Cider House Rules,321,4.10,John Irving,1991,Bantam Books,3.883537
3050,0020869002,Perelandra,422,3.97,C.S. Lewis,1965,Scribner Paper Fiction,3.859328
30675,0140547045,Space Case (Reading Rainbow Book),310,3.89,Edward Marshall,1982,Puffin Books,3.829595
9650,0684823853,THAT HIDEOUS STRENGTH,438,3.87,C.S. Lewis,1996,Scribner,3.828850
24050,0446608904,The Martian Race,626,3.71,Gregory Benford,2001,Aspect,3.769868
12540,0553574019,The Martians,1302,3.53,Kim Stanley Robinson,2000,Spectra Books,3.647340


# Filttrage colaboratif

### 1- Collaborative Filtering Using Knn

In [41]:
# importation de la data

In [42]:
from scipy.sparse import csr_matrix
import glob as g

In [43]:
rating = pd.read_csv('../data_set/BX-CSV-Dump/BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")

In [44]:
books = pd.read_csv('../data_set/BX-CSV-Dump/BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/home/ay/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshe

In [45]:
users = pd.read_csv('../data_set/BX-CSV-Dump/BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")

In [46]:
rating

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [47]:
# je calculer la moyenne d rating des livres pour le  filterage

In [48]:
combiner_livres_rating = pd.merge(rating, books, on='ISBN')

In [49]:
combiner_livres_rating.drop(columns=['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], inplace=True)
combiner_livres_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA


In [50]:
# combiner_livres_rating.drop(columns=['Book-Author', 'Year-Of-Publication', 'Publisher'], inplace=True)

In [51]:
combiner_livres_rating

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA


In [52]:
combiner_livres_rating.isnull().sum()

User-ID                0
ISBN                   0
Book-Rating            0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
dtype: int64

In [53]:
book_ratingCount = (combiner_livres_rating.
                    groupby(by = ['ISBN'])['Book-Rating'].
                    count().
                    reset_index().
                    rename(columns = {'Book-Rating': 'totaleRatingCount'})
                    [['ISBN', 'totaleRatingCount']]
                   )

In [54]:
book_ratingCount

,ISBN,totaleRatingCount
0,0000913154,1
1,0001010565,2
2,0001046438,1
3,0001046713,1
4,000104687X,1
...,...,...
270146,B000234N76,1
270147,B000234NC6,1
270148,B00029DGGO,1
270149,B0002JV9PY,1


In [55]:
rating_with_totalRatingCount = combiner_livres_rating.merge(book_ratingCount, on='ISBN')

In [56]:
rating_with_totalRatingCount

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,totaleRatingCount
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,60
...,...,...,...,...,...,...,...,...
1031131,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,1
1031132,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,1
1031133,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,1
1031134,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,1


In [57]:
# Le livre médian n'a été évalué qu'une seule fois. Regardons le haut de la distribution:

In [58]:
# book_ratingCount['totaleRatingCount'].quantile(np.arange(.9, 1, .01))

In [59]:
# np.arange(.9, 1, .01)

In [60]:
pop_threshold = 100
rating_popular_book = rating_with_totalRatingCount.query('totaleRatingCount >= @pop_threshold ')
rating_popular_book

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,totaleRatingCount
62,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,116
63,278418,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,116
64,638,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,116
65,3363,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,116
66,7158,0446520802,10,The Notebook,Nicholas Sparks,1996,Warner Books,116
...,...,...,...,...,...,...,...,...
521233,263877,0312971125,0,Crazy for You,Jennifer Crusie,2000,St. Martin's Press,111
521234,265115,0312971125,10,Crazy for You,Jennifer Crusie,2000,St. Martin's Press,111
521235,266056,0312971125,0,Crazy for You,Jennifer Crusie,2000,St. Martin's Press,111
521236,267249,0312971125,0,Crazy for You,Jennifer Crusie,2000,St. Martin's Press,111


In [61]:
user_rating = rating_popular_book.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)

In [62]:
user_rating

User-ID,9,14,16,26,39,42,44,51,67,75,...,278800,278807,278813,278819,278828,278832,278836,278843,278844,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [88]:
from fuzzywuzzy import process
from sklearn.neighbors import NearestNeighbors
user_rating_matric = csr_matrix(user_rating.values)
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(user_rating_matric)

NearestNeighbors(algorithm='brute', metric='cosine')

In [64]:
def recommande_knn(titre):
    liste=[]
    titre = titre.lower()
    liste_index = list(user_rating.index)
    liste_index = [item.lower().replace('\\', "") for item in liste_index]
    try:
        idx = liste_index.index(titre)
        distance, index = knn_model.kneighbors(user_rating.iloc[idx, :].values.reshape(1, -1), n_neighbors=100)
        for i in range(1, len(index[0])):
            liste.append(liste_index[index[0][i]])
        liste = [l.lower() for l in liste]
        return liste
    except:
        titre, idx = process.extractOne(titre, list(user_rating.index))
        distance, index = knn_model.kneighbors(user_rating.iloc[idx, :].values.reshape(1, -1), n_neighbors=100)
        for i in range(1, len(index[0])):
            liste.append(liste_index[index[0][i]])
        liste = [l.lower() for l in liste]
        return liste

In [65]:
recommande_knn("4 Blondes")

['diary of a mad bride (summer display opportunity)',
 'mrs dalloway',
 'simple abundance:  a daybook of comfort and joy',
 'message in a bottle',
 'girl, interrupted',
 "angela's ashes: a memoir",
 'code to zero',
 'g is for gumshoe (kinsey millhone mysteries (paperback))',
 'memoirs of a geisha uk',
 'sisterhood of the traveling pants',
 'the virgin suicides',
 'a case of need',
 "schindler's list",
 'little altars everywhere: a novel',
 'seabiscuit',
 "bridget jones's diary",
 "big cherry holler: a big stone gap novel (ballantine reader's circle)",
 'the nanny diaries: a novel',
 "the girls' guide to hunting and fishing",
 "the pilot's wife : a novel",
 'the corrections',
 'bridget jones: the edge of reason',
 'to kill a mockingbird',
 'the reader',
 'atlantis found',
 'dances with wolves',
 'the house of the spirits',
 'while i was gone',
 'here on earth',
 'clear and present danger (jack ryan novels)',
 'beach house',
 'wifey',
 'shopaholic ties the knot',
 'the world according to

### 2- Collaborative Filtering Using Matrix Factorization

In [102]:
from sklearn.decomposition import TruncatedSVD
import warnings
SVD = TruncatedSVD(n_components=12, random_state=17)
X = user_rating.values
matric = SVD.fit_transform(X)
corr = np.corrcoef(matric)
warnings.filterwarnings('ignore', category=RuntimeWarning )

In [115]:
def recommandation_matrix(titre):
    book_title = user_rating.index
    liste_book_title = list(book_title)
    try:
        coffy = liste_book_title.index(titre)
        correlation = corr[coffy]
        liste_ = list(book_title[(correlation>=0.75) ])
        liste_ = [l.lower() for l in liste_]
        return liste_[1:]
    except :
        titre, idx = process.extractOne(titre, list(user_rating.index))
        print(titre)
        correlation = corr[idx]
        liste_ = list(book_title[(correlation>=0.80) ])
        liste_ = [l.lower() for l in liste_]
        return liste_[1:]

In [116]:
recommandation_matrix("4 Blondes")

['a beautiful mind: the life of mathematical genius and nobel laureate john nash',
 'a child called \\it\\": one child\'s courage to survive"',
 'a civil action',
 'a heartbreaking work of staggering genius',
 'a lesson before dying (vintage contemporaries (paperback))',
 "a thousand acres (ballantine reader's circle)",
 "a virtuous woman (oprah's book club (paperback))",
 'about a boy (movie tie-in)',
 "angela's ashes: a memoir",
 'animal dreams',
 'atonement : a novel',
 'atonement: a novel',
 'back roads',
 'bee season: a novel',
 'bel canto: a novel',
 'beloved',
 "big stone gap: a novel (ballantine reader's circle)",
 'breath, eyes, memory',
 'bridget jones: the edge of reason',
 'charming billy',
 'circle of friends',
 'confessions of a shopaholic (summer display opportunity)',
 'confessions of an ugly stepsister : a novel',
 'daisy fay and the miracle man',
 'daughter of fortune',
 'divine secrets of the ya-ya sisterhood : a novel',
 'divine secrets of the ya-ya sisterhood: a no

# Recommandation hybride

In [113]:
def recommandation_hypride(titre):
    livre_mx  = set(recommandation_matrix(titre))
    livre_knn = set(recommande_knn(titre))
    return livre_mx.intersection(livre_knn)

In [114]:
recommandation_hypride("4 Blondes")

{'a heartbreaking work of staggering genius',
 "angela's ashes: a memoir",
 'atonement: a novel',
 'bridget jones: the edge of reason',
 'girl, interrupted',
 'good in bed',
 'jemima j: a novel about ugly ducklings and swans',
 'like water for chocolate: a novel in monthly installments, with recipes, romances, and home remedies',
 'little altars everywhere: a novel',
 'midwives: a novel',
 "she's come undone (oprah's book club (paperback))",
 'sisterhood of the traveling pants',
 "songs in ordinary time (oprah's book club (paperback))",
 'standing in the rainbow',
 'stones from the river',
 'the bean trees',
 "the book of ruth (oprah's book club (paperback))",
 'the corrections',
 'the devil wears prada : a novel',
 "the girls' guide to hunting and fishing",
 'the other boleyn girl',
 'the reader',
 'the virgin blue',
 'the virgin suicides',
 'where the red fern grows',
 'while i was gone',
 "white oleander : a novel (oprah's book club)",
 'wicked: the life and times of the wicked witc